<a href="https://colab.research.google.com/github/research-clone/notebook_tutorials/blob/main/AutoGGUF_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

In [ ]:
# @title # ⚡ AutoGGUF v2

# @markdown Original from AutoGGUF by [@maximelabonne](https://twitter.com/maximelabonne)


# @markdown ❤️ Created by [qnguyen3](https://twitter.com/stablequan).


# @markdown Quantization methods: `q2_k`, `q3_k_l`, `q3_k_m`, `q3_k_s`, `q4_0`, `q4_1`, `q4_k_m`, `q4_k_s`, `q5_0`, `q5_1`, `q5_k_m`, `q5_k_s`, `q6_k`, `q8_0`

# @markdown ---

# @markdown ### ⚡ Quantization parameters

from huggingface_hub import snapshot_download

MODEL_ID = "vilm/Quyen-Pro-v0.1" # @param {type:"string"}
QUANTIZATION_METHODS = "q2_k, q3_k_m, q4_0, q4_k_m, q5_0, q5_k_m, q6_k, q8_0" # @param {type:"string"}
QUANTIZATION_METHODS = QUANTIZATION_METHODS.replace(" ", "").split(",")

# @markdown ### 🤗 Hugging Face Hub
username = "vilm" # @param {type:"string"}
token = "HF_TOKEN" # @param {type:"string"}
MODEL_NAME = MODEL_ID.split('/')[-1]
snapshot_download(repo_id=MODEL_ID, local_dir=MODEL_NAME,
                  local_dir_use_symlinks=False, revision="main")



# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME}_fp16.gguf"
!python llama.cpp/convert-hf-to-gguf.py {MODEL_NAME} --outtype f16 --outfile {fp16}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME}_{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

from huggingface_hub import create_repo, HfApi
from google.colab import userdata, runtime
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Defined in the secrets tab in Google Colab
hf_token = userdata.get(token)
api = HfApi()

# Upload gguf files
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    allow_patterns=["*.gguf","$.md"],
    token=hf_token
)

In [ ]:
!pip install -U transformers